# Instalations

In [1]:
# !pip install psgtray
# !pip install pysimplegui
# !pip install keyboard
# !pip install pyautogui

# Imports

In [2]:
from threading import Thread
import PySimpleGUI as sg
from psgtray import SystemTray
import keyboard
from datetime import datetime
import pyautogui
from time import sleep
import ast
import yaml

# Data Handle

In [3]:
def validate_script(code):
    try:
        ast.parse(code)
    except SyntaxError:
        return False
    
    for line in code.split('\n'):
        if line.startswith('nonlocal'):
            return False
    
    if 'global' in code.split(): 
        return False
    
    return True


# Hotkeys

In [4]:
def hotkey_exec(window, overlay, i):
    window.write_event_value('-HOTKEY-', f'{i}')
    overlay.write_event_value('-HOTKEYSHOW-', f'{i}')
    
def watch_keyboard(window, overlay):
    for i in range(0,10):
        keyboard.add_hotkey(f'ctrl + shift + {i}', hotkey_exec, args =(window, overlay, i))

def reset_keyboard():
    for i in range(0,10):
            keyboard.remove_hotkey(f'ctrl + shift + {i}')

# Snippets Func

In [5]:
add_snippet = lambda key, text: keyboard.add_abbreviation(key, text)
rm_snippet = lambda key: keyboard.remove_abbreviation(e)

# Actions

In [6]:
def exec_hotkeys(keys:str):
    pass

def exec_system_termina(cmd:str):
    pass

def activate_snippets():
    pass

def deactivate_snippets():
    pass



# Mode

In [7]:
class Command(object):
    
    def __init__(self, name:str, hotkey:str, script:str, thumb:str):
        self.__name = name
        self.__hotkey = hotkey
        self.__callback = lambda: exec(script)
        self.__thumb = thumb
        
    
    def call(self):
        self.__callback()
            
            
    def __dict__(self):
        return {
            'name': self.__name,
            'hotkey': self.__hotkey,
            'script': self.__callback,
            'thumb': self.__thumb
        }

class Mode(object):
    def __init__(self, name:str, commands:list):
        self.__name = name
        self.__commands = commands
    
    def __dict__(self):
        return {
            'name': self.__name,
            'commands': [dict(e) for e in self.__commands]
        }

# GUI

In [8]:
def main():

    
    th_detect = {}
    
    # Estado da aplicação
    all_modes = ['Inativo', 'Stream Deck', 'Rodrigo Faro', 'Devil Man']
    cur_mode = 0
    selected_mode = 0
    in_selection = False
    time_last_comand = datetime.now() # Iniciando em uma data específica
    __keep_ths = True
    
    menu = [f'MODO: {cur_mode}', ['Show Window', 'Hide Window', '---', 'Exit']]
    

    tooltip = lambda x = cur_mode: f'Modo: {x}'
    
    def __gen_pad_layout(in_window=True):
    
        def __gen_frame(key, name='BOB', disabled=False):
            nonlocal in_window
            image = sg.Image(
                sg.EMOJI_BASE64_HAPPY_THUMBS_UP, 
                size=(50, 50), pad=(0,0),
                key=f'-THUMB-{name}'
            )
            button = sg.Button(
                'E', pad=(0,0),
                expand_y=True, border_width=0,
                key=f"-EDIT-{name}",
                disabled=disabled
            ) if in_window else False


            label = sg.Text(name, pad=(0,0), expand_x = True, background_color = 'gray')
            frame_layout = [[ image, button ], [label]] if button else [[ image ], [label]]

            return sg.Frame(key, frame_layout, pad=(0,0), key="-FRAME-{Name}")

        buttons_pad = [
            [ __gen_frame('1'), __gen_frame('2'), __gen_frame('3'),],
            [ __gen_frame('4'), __gen_frame('5'), __gen_frame('6'),]
        ]
        
        radio_pad = [
            [sg.Frame('', layout=[[__gen_frame('Esquerda'), __gen_frame('Direita')]], border_width = 0)],
            [__gen_frame('Clique', disabled=True)]


        ]

        pre_layout = []
        
        
        #Layout comum
        pre_layout.append([
            sg.Column(buttons_pad, element_justification='c', pad=(0,0)),
            sg.Column(radio_pad, element_justification='c', pad=(0,0))
        ])

        # Cologando a Layer de informações
        if in_window:  
            pre_layout.append([sg.Multiline(
                expand_x= True, disabled = True, no_scrollbar = True, 
                reroute_cprint = True, size=(0, 3), pad=(0,5),
                key='-INFO-'
            )])
           

        #Selecionaodo cabeçalho a ser usado
        header = [
            sg.Input(disabled = True, pad=(0,0))
        ] if not in_window else [
            sg.Listbox(all_modes, expand_x=True, pad=(0,0))
        ]
        pre_layout.insert(0, header)
        
        return pre_layout

    
    def __create_overlay(
        name:str, layout:list, 
        pos:tuple=(0,0), 
    ):
        nonlocal th_detect

        __xpos, __ypos = pos
        __window = sg.Window( name, layout,
            location=(__xpos, __ypos), 
            keep_on_top=True, auto_size_buttons=False, grab_anywhere=False, return_keyboard_events=False, 
            alpha_channel=0.6, use_default_focus=False, finalize=True, no_titlebar=True )
        __xsize, __ysize = __window.size

        def __is_over_window():
            nonlocal name, __xpos, __ypos, __xsize, __ysize, __window, __keep_ths
            while True:
                if not __keep_ths: break
                x, y = pyautogui.position()
                res = (__ypos < y < (__ypos + __ysize)) * (__xpos < x < (__xpos + __xsize))
                __window.write_event_value(f'-SHOW-', f'{res}')
                sleep(0.3) 

        th = Thread(target=__is_over_window)
        th.start()
        th_detect[name] = th

        return __window

    
    def show_message(text:str):
        nonlocal tray 
        tray.show_message('PyHotKeys', text)
        
        
    def mode_navigate(add:bool):
        nonlocal all_modes, cur_mode, selected_mode
        len_modes = len(all_modes)
        
        selected_mode = selected_mode + 1 if add else selected_mode + 1
        selected_mode = 0  if len_modes < selected_mode + 1 else selected_mode
        selected_mode = len_modes - 1  if selected_mode < 0 else selected_mode
        
        
        show_message(f"ATUAL: {all_modes[cur_mode]}\nSELECIONADO: {all_modes[selected_mode]}")
    

    def __exit():
        nonlocal __keep_ths, overlay, tray, main_window
        __keep_ths = False
        overlay.close()
        tray.close()
        reset_keyboard()
        main_window.close()

        
    #Iniciando telas e observadores
    overlay = __create_overlay('Hotkeys', __gen_pad_layout(False))
    main_window = sg.Window('PyHotKeys', __gen_pad_layout(), finalize=True, enable_close_attempted_event=True)
    watch_keyboard(main_window, overlay)
    tray = SystemTray(menu, single_click_events=False, window=main_window, tooltip=tooltip(), icon=sg.EMOJI_BASE64_HAPPY_JOY)

    
    #Início do processo
    show_message(f'Iniciado, modo no modo: {cur_mode}')
    sg.cprint('iniciadooo')
    while True:
        window, event, values = sg.read_all_windows()
        if event in (sg.WIN_CLOSED,  'Cancel'):
            break

        if event == '-SHOW-':
            val = int(values['-SHOW-'])
            if val:
                window.hide()
            else:
                window.un_hide()
                
                
        if event == '-HOTKEY-':
            cur_time = datetime.now()
            diff_last_time = (cur_time - time_last_comand).total_seconds()
            
            # Bloqueando comandos com menos de 1 segundo de tempo
            time_last_comand = cur_time if diff_last_time > 1 else time_last_comand
            
            
            if values[event] == '0':
                if in_selection:
                    in_selection = False
                    cur_mode = selected_mode
                    show_message(f"Selecionado modo: {all_modes[cur_mode]}")

                else:
                    in_selection = True
                    show_message(f"Entrando em modo de seleção.\nAtualmente em: {all_modes[cur_mode]}")


            if values[event] == '8':
                if in_selection == True:
                    mode_navigate(False)

            if values[event] == '9':
                if in_selection == True:
                    mode_navigate(True)


            if values[event].isnumeric() and not values[event] in '89':
                if in_selection == False:
                    print('ok')
                        
        
        if event in ('Show Window', sg.EVENT_SYSTEM_TRAY_ICON_DOUBLE_CLICKED):
            window.un_hide()
            window.bring_to_front()
        elif event in ('Hide Window', sg.WIN_CLOSE_ATTEMPTED_EVENT):
            window.hide()
            tray.show_icon()


            
    #Matando processos
    __exit()
        

    
    
    
if __name__ == '__main__':
    main()